In [1]:
import os
import sys
import requests

import numpy as np
import pandas as pd

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [2]:
np.random.seed(42)


In [3]:
df_path = os.path.join(os.getcwd(), os.pardir, 'data', 'data.csv')
df = pd.read_csv(df_path)

features = ['acousticness', 'danceability','energy',
               'instrumentalness', 'key', 'liveness', 'loudness',
               'mode','speechiness', 'tempo',
               'valence']
df_train = df[features]

df_train.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.995,0.708,0.1950,0.563,10,0.1510,-12.428,1,0.0506,118.469,0.7790
1,0.994,0.379,0.0135,0.901,8,0.0763,-28.454,1,0.0462,83.972,0.0767
2,0.604,0.749,0.2200,0.000,5,0.1190,-19.924,0,0.9290,107.177,0.8800
3,0.995,0.781,0.1300,0.887,1,0.1110,-14.734,0,0.0926,108.003,0.7200
4,0.990,0.210,0.2040,0.908,11,0.0980,-16.829,1,0.0424,62.149,0.0693


In [4]:
scaler = StandardScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train),
                               columns=features)
df_train_scaled.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,1.332319,0.968662,-1.097999,1.296562,1.365333,-0.314998,-0.186652,0.641344,-0.289840,0.049500,0.940924
1,1.329664,-0.907636,-1.776785,2.389253,0.796383,-0.737519,-3.014729,0.641344,-0.319186,-1.073199,-1.735454
2,0.294154,1.202486,-1.004503,-0.523513,-0.057043,-0.495997,-1.509457,-1.559227,5.568626,-0.317996,1.325822
3,1.332319,1.384983,-1.341091,2.343994,-1.194943,-0.541247,-0.593587,-1.559227,-0.009722,-0.291114,0.716082
4,1.319044,-1.871449,-1.064341,2.411883,1.649808,-0.614778,-0.963288,0.641344,-0.344530,-1.783425,-1.763655


### Autoencoder

In [11]:
n = df_train_scaled.shape[1]

# Encoder

encoder = Sequential([Dense(n // 2, name='encode_2', input_shape=(n,))])

encoder.compile(optimizer='adam', loss='mse')
encoder.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encode_2 (Dense)             (None, 5)                 60        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Decoder
decoder = Sequential([Dense(n, name='decode_2', input_shape=(n // 2,))])

decoder.compile(optimizer='adam', loss='mse')
decoder.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decode_2 (Dense)             (None, 11)                66        
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


### Build the full autoencoder and train it

In [14]:
input_layer = Input(shape=(n,))
encoder_output = encoder(input_layer)
decoder_output = decoder(encoder_output)
autoencoder = Model(input_layer, decoder_output)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 5)                 60        
_________________________________________________________________
sequential_6 (Sequential)    (None, 11)                66        
Total params: 126
Trainable params: 126
Non-trainable params: 0
_________________________________________________________________


In [15]:
stop = EarlyStopping(monitor='loss', 
                     patience=5, 
                     restore_best_weights=True)

history = autoencoder.fit(df_train_scaled, 
                          df_train_scaled, 
                          epochs=20, 
                          batch_size=128,
                          callbacks=[stop])

Train on 169909 samples
Epoch 1/20
169909/169909 [==============================] - 4s 23us/sample - loss: 0.5066
Epoch 2/20
169909/169909 [==============================] - 2s 13us/sample - loss: 0.3221
Epoch 3/20
169909/169909 [==============================] - 2s 13us/sample - loss: 0.3083
Epoch 4/20
169909/169909 [==============================] - 2s 12us/sample - loss: 0.3064
Epoch 5/20
169909/169909 [==============================] - 2s 13us/sample - loss: 0.3056
Epoch 6/20
169909/169909 [==============================] - 2s 12us/sample - loss: 0.3048
Epoch 7/20
169909/169909 [==============================] - 2s 13us/sample - loss: 0.3043
Epoch 8/20
169909/169909 [==============================] - 2s 12us/sample - loss: 0.3039
Epoch 9/20
169909/169909 [==============================] - 2s 13us/sample - loss: 0.3038
Epoch 10/20
169909/169909 [==============================] - 2s 13us/sample - loss: 0.3037
Epoch 11/20
169909/169909 [==============================] - 2s 13us/sample

### Save the trained models

In [16]:
encoder.save('encoder.h5')
decoder.save('decoder.h5')
autoencoder.save('autoencoder.h5')

### Build an end to end model with encoder and nearest neighbors

We are stacking a keras NN encoder on top of a scikit-learn nearest neighbors model. The former reduces the dimensionality, from 11 to 5 features. 

In [31]:
class End2EndModel():
    def __init__(self, n_examples, encoder_file):
        self.scaler = StandardScaler()
        self.encode = load_model(encoder_file)

        self.nearest_n = NearestNeighbors(n_examples)
        
    def fit(self, X):
        X_scaled = self.scaler.fit_transform(X)
        encoded = self.encode.predict(X_scaled)
        nn = self.nearest_n.fit(encoded)
        return nn
    
    def predict(self, x):
        x_scaled = self.scaler.transform(x)
        encoded = self.encode.predict(x_scaled)
        scores, indices = self.nearest_n.kneighbors(encoded)
        return scores, indices
                

In [32]:
model = End2EndModel(10, 'encoder.h5')

C:\Users\A\AppData\Local\Continuum\anaconda3\envs\song-recommend\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [33]:
model.fit(df_train)

NearestNeighbors(n_neighbors=10)

### Try it out with an item from the dataset

In [34]:
test = np.array(df_train.iloc[3764])
test = test.reshape(1, -1)
test


array([[ 1.3900e-01,  3.1600e-01,  7.7200e-01,  5.3200e-01,  1.1000e+01,
         2.3800e-01, -1.1605e+01,  0.0000e+00,  6.4900e-02,  8.8648e+01,
         6.6000e-01]])

In [36]:
scores, indices = model.predict(test)
scores, indices

(array([[8.42936970e-08, 3.09293462e-01, 3.32585634e-01, 4.00838659e-01,
         4.18509141e-01, 4.19979617e-01, 4.35221213e-01, 5.10055541e-01,
         5.10543625e-01, 5.13623751e-01]]),
 array([[  3764,   4432, 125668, 149375,  55051,  73453,  46726,  46445,
           5257, 151981]], dtype=int64))

### Retrieve our suggestions

In [37]:
results = []
for i, index in enumerate(indices[0]):
    track_id = df.iloc[index]['id']
    artists = df.iloc[index]['artists']
    title = df.iloc[index]['name']
    results.append({'index': index,
                    'track_id': track_id,
                    'artists': artists,
                    'title': title,
                    'score': scores[0][i]})
    
result_table = pd.DataFrame(results)
result_table.sort_values(by='score')

,index,track_id,artists,title,score
0,3764,1qPSxRyMfES52PbpxCzWcd,['Al Di Meola'],Race With Devil On Spanish Highway,8.429370e-08
1,4432,0lFQqCxJiT0pvUPms0bzSg,['Thin Lizzy'],Thunder and Lightning,3.092935e-01
2,125668,7g3htkaLz4ETFn0cifwM3y,['Lana Del Rey'],High By The Beach,3.325856e-01
3,149375,7cophW0FqeDEEW9i57WBvL,['Whitesnake'],Love Ain't No Stranger,4.008387e-01
4,55051,0M8MPqaoGgkRjkG9kgZ3n4,"['Justine Skye', 'Tyga']",Collide (feat. Tyga),4.185091e-01
5,73453,2MOurmmosbkle2QinkSZAz,['Willie Hutch'],Out There,4.199796e-01
6,46726,57HwKH3pHLeelTkckr94qf,['The Dodos'],Horny Hippies,4.352212e-01
7,46445,3oLullgZ9FsDo6O93nppku,['Underoath'],"Some Will Seek Forgiveness, Others Escape",5.100555e-01
8,5257,2PAol2oDdGSHys8hc0gtLX,['Tori Amos'],Precious Things,5.105436e-01
9,151981,26gexilnrxUTfBhiudNBrT,['Los Temerarios'],Te Hice Mal,5.136238e-01


### Explore ways to persist the scikit-learn layer

In [45]:
import pickle
with open('nearest.pickle', 'wb') as f:
    pickle.dump(model.nearest_n, f)

In [46]:
from joblib import dump, load
dump(model.nearest_n, 'nearest.joblib')

['nearest.joblib']